# Correlação dos termos de erro


Neste notebook irei ilustrar, a partir de um dataset de dados de consumo de usuários, o impacto de registros duplicados na determinação dos coeficientes da regressão

## 1. Configurações do ambiente e importações

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import norm
import statsmodels.api as sm

In [3]:
%config InlineBackend.figure_format = 'retina'

In [7]:
# Configurações gerais
np.random.seed(42)
n = 100           # Número de usuários originais
beta_age = 0.3    # Coeficiente para idade
beta_hours = 0.7  # Coeficiente para horas jogadas
intercept = 2.0   # Intercepto
sigma = 1.5       # Desvio padrão dos erros

# Gerar variáveis independentes
ages = np.random.randint(18, 50, n)  # Idades dos usuários
hours_played = np.random.uniform(1, 10, n)  # Horas jogadas semanalmente

# Gerar a variável dependente (satisfação do jogo)
errors = np.random.normal(0, sigma, n)
satisfaction = intercept + beta_age * ages + beta_hours * hours_played + errors

# Criar o DataFrame original
data_original = pd.DataFrame({
    "Age": ages,
    "HoursPlayed": hours_played,
    "Satisfaction": satisfaction
})

In [11]:
# Duplicar os dados (50% duplicado)
data_duplicated = pd.concat([data_original, data_original.sample(frac=1, random_state=42)], ignore_index=True)

In [12]:
# Ajustar os modelos nos dois casos
X_original = sm.add_constant(data_original[["Age", "HoursPlayed"]])
model_original = sm.OLS(data_original["Satisfaction"], X_original).fit()

X_duplicated = sm.add_constant(data_duplicated[["Age", "HoursPlayed"]])
model_duplicated = sm.OLS(data_duplicated["Satisfaction"], X_duplicated).fit()

In [21]:
# Exibir os resultados dos coeficientes e intervalos de confiança
results = pd.DataFrame({
    "SE Original": model_original.bse,
    "SE Duplicado": model_duplicated.bse,
    "t-Statistic Original": model_original.tvalues,
    "t-Statistic Duplicado": model_duplicated.tvalues,
    "IC Inferior Original": model_original.conf_int()[0],
    "IC Inferior Duplicado": model_duplicated.conf_int()[0],
    "IC Superior Original": model_original.conf_int()[1],
    "IC Superior Duplicado": model_duplicated.conf_int()[1],
    "p-Value Original": model_original.pvalues,
    "p-Value Duplicado": model_duplicated.pvalues
})

In [23]:
results

,SE Original,SE Duplicado,t-Statistic Original,t-Statistic Duplicado,IC Inferior Original,IC Inferior Duplicado,IC Superior Original,IC Superior Duplicado,p-Value Original,p-Value Duplicado
const,0.650189,0.456239,4.753886,6.774792,1.800479,2.191185,4.381370,3.990664,6.928863e-06,1.405262e-10
Age,0.015938,0.011184,17.465121,24.889653,0.246724,0.256301,0.309988,0.300411,1.042026e-31,9.793366e-63
HoursPlayed,0.054939,0.038551,12.151799,17.317604,0.558570,0.591583,0.776648,0.743634,3.363840e-21,1.926893e-41
